# Ligand-based experiments to predict binding affinities in pkis2

This notebook featurizes the pkis2 dataset with `OneHotSMILESFeaturizer` and provides in-disk `npz` files for each kinase and measurement type.

Output files are written to `_output/`, as in:

* `_output/PKIS2__YES__PercentageDisplacementMeasurement.npz`
* `_output/PKIS2__YSK1__PercentageDisplacementMeasurement.npz`
* `_output/PKIS2__YSK4__PercentageDisplacementMeasurement.npz`
* `_output/PKIS2__ZAK__PercentageDisplacementMeasurement.npz`


Each `npz` will contain two `np.ndarray` objects: `X` (featurized systems) and `y` (associated measurements), plus the train/test/validation indices.

In [24]:
# Filter out some warnings thrown by openforcefield and rdkit
import warnings
warnings.simplefilter("ignore")
import logging
logging.basicConfig(level=logging.ERROR)

import numpy as np
import os
from pathlib import Path

import pytorch_lightning as pl
pl.seed_everything(1234);

In [25]:
HERE = Path(_dh[-1])

In [26]:
from kinoml.datasets.kinomescan.pkis2 import PKIS2DatasetProvider
pkis2 = PKIS2DatasetProvider.from_source()

In [27]:
pkis2

<PKIS2DatasetProvider with 261870 PercentageDisplacementMeasurement measurements and 257920 systems (AminoAcidSequence=403, SmilesLigand=640)>

In [28]:
df = pkis2.to_dataframe()
df

,Systems,n_components,PercentageDisplacementMeasurement
0,AAK1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C2...,2,14.0
1,ABL1-nonphosphorylated & Clc1cccc(Cn2c(nn3c2nc...,2,28.0
2,ABL1-nonphosphorylated & Clc1cccc(Cn2c(nn3c2nc...,2,20.0
3,ABL2 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C2...,2,5.0
4,ACVR1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C...,2,0.0
...,...,...,...
261865,ZAP70 & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)cc12)...,2,0.0
261866,p38-alpha & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)c...,2,0.0
261867,p38-beta & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)cc...,2,0.0
261868,p38-delta & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)c...,2,0.0


This featurization pipeline consists of:

- Converting the OFF molecule to RDKit molecule
- Generating the one-hot encoding of a SMILES, padded to the maximum length.

In [29]:
from kinoml.features.ligand import SmilesToLigandFeaturizer, OneHotSMILESFeaturizer
from kinoml.features.protein import AminoAcidCompositionFeaturizer
from kinoml.features.core import ScaleFeaturizer, Concatenated, Pipeline, PadFeaturizer

onehot_smiles_featurizer = Pipeline([SmilesToLigandFeaturizer(style="rdkit"), OneHotSMILESFeaturizer()])

In [30]:
# Consider only the 10 first systems
pkis2 = pkis2.__class__(pkis2.measurements[:10])

In [31]:
# prefeaturize everything -- use single process for this dataset to benefit from the LRU cache!
pkis2.featurize(onehot_smiles_featurizer, processes=1);

Remove systems that couldn't be featurized

In [32]:
from kinoml.datasets.groups import CallableGrouper, RandomGrouper
grouper = CallableGrouper(lambda measurement: 'invalid' if 'last' not in measurement.system.featurizations else 'valid')
grouper.assign(pkis2, overwrite=True)
groups = pkis2.split_by_groups()
len(groups.get('valid', [])), len(groups.get('invalid', []))

(0, 10)

Split by kinase name, since these models are ligand-based (so one model per kinase).

In [11]:
grouper = CallableGrouper(lambda measurement: measurement.system.protein.name)
grouper.assign(groups['valid'], overwrite=True)
groups_by_kinase = groups['valid'].split_by_groups()

Split each kinase group by measurement type too. We need to for-loops for that:

In [12]:
type_grouper = CallableGrouper(lambda measurement: type(measurement).__name__)
random_grouper = RandomGrouper({"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1})

output = HERE / "_output/"
output.mkdir(parents=True, exist_ok=True)

for kinase, ds in sorted(groups_by_kinase.items(), key=lambda kv: len(kv[1]), reverse=True):
    type_grouper.assign(ds, overwrite=True)
    types = ds.split_by_groups()
    for mtype, ds_ in types.items():
        indices = random_grouper.indices(ds)
        X = np.asarray(ds_.featurized_systems())
        y = ds_.measurements_as_array()
        np.savez(output / f"PKIS2__{kinase}__{mtype}.npz", X=X, y=y.astype('float32'), **indices)

Annotate observation models for `pytorch` and `xgboost` (we will need this in next notebooks)

In [13]:
observation_model_pytorch = pkis2.observation_model(backend="pytorch")
display(observation_model_pytorch)

<function kinoml.core.measurements.PercentageDisplacementMeasurement._observation_model_pytorch(dG_over_KT, inhibitor_conc=1, standard_conc=1, **kwargs)>

# Reproducibility logs

In [14]:
from kinoml.utils import watermark
watermark()

Watermark
---------
Last updated: 2020-12-03T18:45:12.289155+01:00

Python implementation: CPython
Python version       : 3.7.8
IPython version      : 7.19.0

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Hostname: Talias-MacBook-Pro.local

Git hash: ab848d47088e039875c021ea989b18861b5f2948

numpy            : 1.19.4
pytorch_lightning: 1.0.8
logging          : 0.5.1.2
kinoml           : 0+unknown

Watermark: 2.1.0


conda
-----
sys.version: 3.7.7 (default, Mar 23 2020, 17:31:31) 
...
sys.prefix: /opt/miniconda3
sys.executable: /opt/miniconda3/bin/python
conda location: /opt/miniconda3/lib/python3.7/site-packages/conda
conda-build: None
conda-env: /opt/miniconda3/bin/conda-env
user site dirs: 

CIO_TEST: <not set>
CONDA_DEFAULT_ENV: kinoml-pinned-by-commit
CONDA_EXE: /opt/miniconda3/bin/conda
CONDA_MKL_INTERFACE_LAYER_BACKUP: 
CONDA_PREFIX: /opt/miniconda3/envs/kinoml-pinned-by-commit
CO